In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
# Load the annotations from a CSV file
annotations_df = pd.read_csv('/kaggle/input/my-other-data/train/_annotations.csv')
annotations_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Tomato-leaf-bacterial-spot-20-_jpg.rf.7db1aab3...,640,640,Tomato leaf baterial spot,0,0,320,297
1,Tomato-leaf-bacterial-spot-20-_jpg.rf.7db1aab3...,640,640,Tomato leaf baterial spot,0,373,319,640
2,Tomato-leaf-bacterial-spot-20-_jpg.rf.7db1aab3...,640,640,Tomato leaf baterial spot,321,311,602,640
3,Tomato-leaf-late-blight-15-_jpg.rf.7dcf2d8ae84...,640,640,Tomato leaf late blight,7,0,638,634
4,39tomato_early_blight_jpg.rf.7dfa2f4a98e65481d...,640,640,EarlyBlight,145,275,193,354


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

from sklearn.model_selection import train_test_split
# Define the input shape of the images
img_width = 224
img_height = 224
input_shape = (img_width, img_height, 3)

# Define the batch size
batch_size = 36

# Load the annotations from a CSV file
annotations_df = pd.read_csv('/kaggle/input/my-other-data/train*  /_annotations.csv')

# Split the dataset into train and validation sets
train_df, valid_df = train_test_split(annotations_df, test_size=0.2, random_state=None)

# Create an instance of the ImageDataGenerator class to perform data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Load the training dataset using the flow_from_dataframe method
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='/kaggle/input/my-other-data/train/',
        x_col='filename',
        y_col='class',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        interpolation='bicubic',
        shuffle=True,
        seed=123)

# Create an instance of the ImageDataGenerator class for the validation dataset
valid_datagen = ImageDataGenerator(rescale=1./255)
# Load the validation dataset using the flow_from_dataframe method
valid_generator = valid_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='/kaggle/input/my-other-data/train/',
        x_col='filename',
        y_col='class',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        interpolation='bicubic',
        shuffle=True,
        seed=123)

# Define the CNN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(13, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=15,
        validation_data=valid_generator,
        validation_steps=valid_generator.samples // batch_size,
        workers=tf.data.AUTOTUNE,
        use_multiprocessing=True)

# Evaluate the model
model.evaluate(valid_generator)
model.save("my_model.h5")

Found 10284 validated image filenames belonging to 13 classes.
Found 2572 validated image filenames belonging to 13 classes.
Epoch 1/15
285/285 [==============================] - 278s 942ms/step - loss: 1.8599 - accuracy: 0.3864 - val_loss: 1.1555 - val_accuracy: 0.6264
Epoch 2/15
285/285 [==============================] - 259s 911ms/step - loss: 0.9616 - accuracy: 0.7142 - val_loss: 0.7259 - val_accuracy: 0.7817
Epoch 3/15
285/285 [==============================] - 259s 908ms/step - loss: 0.6330 - accuracy: 0.8129 - val_loss: 0.5410 - val_accuracy: 0.8650
Epoch 4/15
285/285 [==============================] - 254s 891ms/step - loss: 0.4804 - accuracy: 0.8505 - val_loss: 0.4792 - val_accuracy: 0.8584
Epoch 5/15
285/285 [==============================] - 256s 899ms/step - loss: 0.4083 - accuracy: 0.8733 - val_loss: 0.3514 - val_accuracy: 0.8959
Epoch 6/15
285/285 [==============================] - 264s 927ms/step - loss: 0.3437 - accuracy: 0.8855 - val_loss: 0.3516 - val_accuracy: 0.8908

In [11]:
loaded_model = tf.keras.models.load_model('/kaggle/working/my_model.h5')
loaded_model.layers[0].input_shape #(None, 160, 160, 3)

(None, 224, 224, 3)

In [13]:
import tensorflow as tf

model = tf.keras.models.load_model('/kaggle/working/my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("another_model.tflite", "wb").write(tflite_model)

22256940

In [15]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path='/kaggle/working/another_model.tflite')


In [17]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the model from the .h5 file
model = load_model('/kaggle/working/my_model.h5')

# Load an example image
img_path = '/kaggle/input/my-keras-data/test/102Tomato_late_blight_jpg.rf.febbe89bbb4360f0788434dcf1d4c3c7.jpg'
img = load_img(img_path, target_size=(224, 224))

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Make a prediction with the model
prediction = model.predict(img_array)

# Print the predicted class and its probability
class_index = np.argmax(prediction)
class_prob = prediction[0, class_index]
print(f'Predicted class index: {class_index}')
print(f'Predicted class probability: {class_prob *100} ')


1/1 [==============================] - 0s 138ms/step
Predicted class index: 2
Predicted class probability: 48.65232706069946 


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/kaggle/working/my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("another_model.tflite", "wb").write(tflite_model)